In [ ]:
import os
import sys
import json
import argparse
from typing import *
from tqdm import *
import numpy as np
import matplotlib.pyplot as plt

sys.path.append("..")
try:
  from ..utils.logger import logger
  from ..utils.utils import *
except ImportError:
  from utils.logger import logger
  from utils.utils import *

In [ ]:
import numpy as np
from scipy import integrate
#import matplotlib.pyplot as plt
import sympy

def apply_ics(sol, ics, x, known_params):
	free_params = sol.free_symbols - set(known_params)
    ## free parameters like C1,C2, that needed to be figured out with ics and boundary conditions
	eqs = [(sol.lhs.diff(x, n) - sol.rhs.diff(x, n)).subs(x, 0).subs(ics) for n in range(len(ics))]
    ## form equations with general solution(sol), by substituting the x with 0 and [y(0),dy/dx(x=0)...] with ics.
	sol_params = sympy.solve(eqs, free_params)
    ## solve the algebraic equation set  to get the free parameters expression
    ## return the solution after substituting the free parameters

	return sol.subs(sol_params)

sympy.init_printing()
## initialize the print environment
t = sympy.Symbol("t")
## symbolize the parameters and they can only be positive
x1 = sympy.Function('x_1')
x2 = sympy.Function('x_2')
## set x to be the differential function, not the variable
f1 = 80000 * (x1(t) - x2(t)) + 10000 * (x1(t).diff(t) - x2(t).diff(t)) - x2(t).diff(t, 2) * 2433
f2 = 80000 * (x1(t) - x2(t)) + 10000 * (x1(t).diff(t) - x2(t).diff(t)) - 6250 * sympy.cos(1.0045 * t) * x2(t).diff(t, 2) + \
        1e3 * 9.8 * sympy.pi * x1(t) + 656.3616 * x1(t).diff(t) + 1335.535 * x1(t).diff(t, 2) + 4866 * x1(t).diff(t, 2)
sol = sympy.dsolve(f1, f2)
## use diff() and dsolve to get the general solution
ics = {x1(0): 0, x1(t).diff(t).subs(t, 0): 0, x2(0): 0, x2(t).diff(t).subs(t, 0): 0}
## apply dict to the initial conditions
t_sol = apply_ics(solsol, ics, t, [])
## get the solution with ics by calling function apply_ics.
sympy.pprint(t_sol)
print(t_sol)
## pretty print
## solution is as follows

In [1]:
from wolframclient.evaluation import WolframLanguageSession
from wolframclient.language import wl, wlexpr
session = WolframLanguageSession()

In [ ]:
session.evaluate("""
M = 4866; m = 2433; k2 = 80000; c2 = 10000; omega = 2.2143; k1 = 1025 * 9.8 * Pi;
f = 4890; c1 = 167.8395; m0 = 1165.992;
start = 510; stop = 520; dim = 1;
P2[cm2_] := Sum[cm2 * ((x1'[i] - x2'[i])^2) * dim, {i, start, stop, dim}] /. 
  NDSolve[{m*x2''[t]==k2*(x1[t]-x2[t])+cm2*(x1'[t]-x2'[t]), 
    f*Cos[omega * t]==k2*(x1[t]-x2[t])+cm2*(x1'[t]-x2'[t])+k1*x1[t]+c1*x1'[t]+m0*x1''[t]+M*x1''[t], 
    x1[0]==x2[0]==x1'[0]==x2'[0]==0},
  {x1, x2}, {t, start, stop}]""")

In [3]:
def PP2(x, d):
  return session.evaluate(wl.Evaluate(wlexpr(f'First[P2[{x}]-P2[{x-d}]]')))

0.1465828107559073
<class 'float'>


In [9]:
def find(start, stop, d, f, esp=1e-4):
  print(f"find({start}, {stop}, {d})")
  m = (start + stop) / 2
  v = f(m, d)
  if abs(v) < esp:
    return [m, v]
  if v < 0:
    return find(start, m, d, f)
  return find(m, stop, d, f)

find(0, 100000, 1, PP2)

find(0, 100000, 1)
find(0, 50000.0, 1)
find(25000.0, 50000.0, 1)
find(25000.0, 37500.0, 1)
find(31250.0, 37500.0, 1)
find(34375.0, 37500.0, 1)
find(34375.0, 35937.5, 1)
find(34375.0, 35156.25, 1)
find(34375.0, 34765.625, 1)
find(34375.0, 34570.3125, 1)
find(34375.0, 34472.65625, 1)
find(34375.0, 34423.828125, 1)
find(34399.4140625, 34423.828125, 1)
find(34411.62109375, 34423.828125, 1)
find(34417.724609375, 34423.828125, 1)
find(34420.7763671875, 34423.828125, 1)
find(34422.30224609375, 34423.828125, 1)
find(34422.30224609375, 34423.065185546875, 1)
find(34422.68371582031, 34423.065185546875, 1)


[34422.874450683594, 8.194906013159198e-05]